In [1]:
#PIP INSTALLS:
!pip install wikipedia keybert sentence-transformers transformers spacy datasets evaluate matplotlib accelerate nltk torch
!python -m spacy download en_core_web_sm
!python -m nltk.downloader averaged_perceptron_tagger

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#IMPORTS:
import nltk
from itertools import product
import wikipedia
from sentence_transformers import SentenceTransformer, util, InputExample, losses, models
import re
import json
import spacy
from keybert import KeyBERT
import re
import logging
from transformers import pipeline
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, accuracy_score
from google.colab import drive, files
from datasets import load_dataset, Dataset
from tqdm import tqdm
import evaluate
import matplotlib.pyplot as plt
from spacy.lang.en.stop_words import STOP_WORDS
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import default_data_collator
drive.mount("/content/drive")
nltk.download('averaged_perceptron_tagger_eng')
nlp = spacy.load("en_core_web_sm")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [3]:
# 3. If 'QA' is inside 'Distil-BERT', run this
!ls "/content/drive/My Drive/Distil-BERT/1/QA/SentenceTransformer"

config.json			   README.md		      tokenizer.json
config_sentence_transformers.json  sentence_bert_config.json  vocab.txt
model.safetensors		   special_tokens_map.json
modules.json			   tokenizer_config.json


In [4]:
#DEFINITIONS:
EXCLUDE_TAGS = {"IN", "CC"}
# Common slang dictionary
SLANG_DICT = {
    "u": "you", "ur": "your", "r": "are", "y": "why",
    "wat": "what", "wats": "what is", "wanna": "want to",
    "gonna": "going to", "tho": "though", "pls": "please",
    "thx": "thanks", "lol": "", "omg": "oh my god",
    "idk": "I don't know", "btw": "by the way", "smh": "shaking my head",
    "bff": "best friend forever", "brb": "be right back", "tbh": "to be honest",
    "fyi": "for your information", "lmao": "laughing my ass off", "rofl": "rolling on the floor laughing",
    "ttyl": "talk to you later", "np": "no problem", "yolo": "you only live once",
    "fam": "family", "lit": "exciting or excellent", "savage": "ruthless or harsh",
    "slay": "to do something exceptionally well", "bae": "before anyone else", "cray": "crazy",
    "fomo": "fear of missing out", "swag": "style or coolness", "vibe": "mood or atmosphere",
    "tbh": "to be honest", "imo": "in my opinion", "srsly": "seriously", "jk": "just kidding",
    "ppl": "people", "wyd": "what you doing", "smexy": "sexy", "af": "as f*ck",
    "wth": "what the hell", "wtf": "what the f*ck", "asl": "age, sex, location",
    "ik": "I know", "ikr": "I know, right?", "lml": "laughing madly", "hmu": "hit me up",
    "mfw": "my face when", "fml": "f*ck my life", "yass": "yes"
}

def is_permutable(pos_tag):
    return pos_tag not in EXCLUDE_TAGS

def generate_cased_variants(words, tags):
    options = []

    for word, tag in zip(words, tags):
        if word and word[0].isalpha() and is_permutable(tag):
            # Only change the first character
            lower = word[0].lower() + word[1:]
            upper = word[0].upper() + word[1:]
            options.append([lower, upper])
        else:
            options.append([word])

    # Generate all combinations
    return [' '.join(p) for p in product(*options)]

# Test function
def case_variants(text):
    words = text.split()
    tags = [tag for _, tag in nltk.pos_tag(words)]
    return generate_cased_variants(words, tags)

def extract_keyword_spacy_keybert(question):
    noun_phrases = []
    for question in case_variants(question):
        doc = nlp(question)
        question_lower = question.lower()

        priority_entity_labels = ["PERSON", "ORG", "GPE", "FAC", "LOC"]
        named_entities = [ent.text for ent in doc.ents if ent.label_ in priority_entity_labels]

        if named_entities:
            return named_entities[0]

        noun_phrases += [chunk.text for chunk in doc.noun_chunks]

    keywords = kw_model.extract_keywords(question.title(), top_n=3, stop_words='english')
    bert_keywords = [kw[0] for kw in keywords]


    candidates = list(dict.fromkeys(noun_phrases + bert_keywords))

    filtered_candidates = [
        c for c in candidates
        if c.lower() not in STOP_WORDS and len(c.strip()) > 2]


    filtered_candidates = sorted(filtered_candidates, key=lambda x: question_lower.find(x.lower()))

    return filtered_candidates[0] if filtered_candidates else question

#alternative technique
def get_wikipedia_article(query):
    try:

        try:
            page = wikipedia.page(query, auto_suggest=False)
            print(f"Retrieved (exact match): {page.title}")
            return page.content
        except (wikipedia.DisambiguationError, wikipedia.PageError):
            pass


        search_results = wikipedia.search(query)
        if not search_results:
            print("No search results found.")
            return None


        for title in search_results:
            try:
                page = wikipedia.page(title)
                if "replica" not in page.title.lower():
                    print(f"Retrieved: {page.title}")
                    return page.content
            except (wikipedia.DisambiguationError, wikipedia.PageError):
                continue

        print("No valid Wikipedia page found.")
        return None

    except Exception as e:
        print(f"Error occurred: {e}")
        return None


def get_best_wikipedia_article(question, top_n=5):
    # Step 1: Search using the full question
    search_results = wikipedia.search(question, results=top_n)


    print("🔎 Candidates:", search_results)
    # Step 2: Get summaries of each
    summaries = []
    titles = []
    for title in search_results:
        try:
            summary = wikipedia.summary(title, sentences=3)
            summaries.append(summary)
            titles.append(title)
        except wikipedia.exceptions.DisambiguationError:
            continue
        except wikipedia.exceptions.PageError:
            continue

    if not summaries:
        return None
    case_vars = case_variants(question)
    summary_embeddings = embedder.encode(summaries, convert_to_tensor=True)
    cosine_scores = []
    for question in case_vars:
        # Step 3: Use SentenceTransformer to rank similarity
        question_embedding = embedder.encode(question, convert_to_tensor=True)
        cosine_scores.append(util.cos_sim(question_embedding, summary_embeddings))
    score_matrix = torch.vstack(cosine_scores)  # shape: [V x N]
    best_scores, _ = score_matrix.max(dim=0)
    best_idx = best_scores.argmax().item()
    best_title = titles[best_idx]
    best_page = wikipedia.page(best_title)

    print(f"✅ Best match: {best_title}")
    return best_page.content, best_title

def expand_question(question):
    """Replace common slangs and shorthand in questions."""
    words = question.lower().split()
    expanded = [SLANG_DICT.get(w, w) for w in words]
    return ' '.join(expanded)

def clean_text(text):
    """Remove unwanted characters and normalize spacing."""
    text = re.sub(r'\[\d+\]', '', text)  # Remove references like [1]
    return re.sub(r'\s+', ' ', text).strip()

#alternative technique
def chunk_text_by_tokens(text, max_tokens=450):
    """Split long text into chunks based on token length."""
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

def chunk_text_with_stride(text, max_length=384, stride=128):
    inputs = tokenizer(text, return_overflowing_tokens=True, max_length=max_length, stride=stride, truncation=True, padding="max_length")
    return [tokenizer.decode(input_ids, skip_special_tokens=True) for input_ids in inputs["input_ids"]]


def rank_chunks_semantic(question, chunks, top_k=3):
    question_emb = embedder.encode(question, convert_to_tensor=True)
    chunk_embs = embedder.encode(chunks, convert_to_tensor=True)
    scores = util.cos_sim(question_emb, chunk_embs)[0]
    top_indices = scores.argsort(descending=True)[:top_k]
    return [chunks[i] for i in top_indices]

#alternative technique
def rank_chunks_tfidf(question, chunks, top_k=3):
    """Rank text chunks based on TF-IDF similarity to the question."""
    vectorizer = TfidfVectorizer().fit([question] + chunks)
    vectors = vectorizer.transform([question] + chunks)
    scores = cosine_similarity(vectors[0:1], vectors[1:]).flatten()
    top_indices = scores.argsort()[-top_k:][::-1]
    return [chunks[i] for i in top_indices]

def is_valid_answer(answer_text, score, min_len=3, min_score=0.1):
    """Check if the answer is valid based on length and confidence."""
    return len(answer_text.strip()) >= min_len and score >= min_score

def get_best_answer(question, text_chunks):
    """Run QA model on ranked chunks and return best answer."""
    answers = []
    for chunk in text_chunks:
        try:
            result = qa_pipeline({'question': question, 'context': chunk})
            if is_valid_answer(result['answer'], result['score']):
                cleaned = re.sub(r'[^\w\s,.]', '', result['answer']).strip()
                answers.append((cleaned, result['score']))
        except Exception as e:
            logger.warning(f"QA failed on chunk: {e}")
    return max(answers, key=lambda x: x[1])[0] if answers else "No good answer found."

def answer_question(question, full_text):
    """Top-level QA function: cleans, expands, chunks, ranks, answers."""
    question_expanded = expand_question(question.lower())
    cleaned_text = clean_text(full_text)
    chunks = chunk_text_with_stride(cleaned_text)
    top_chunks = rank_chunks_semantic(question_expanded, chunks)
    return get_best_answer(question_expanded, top_chunks)

def is_short_answer(example, max_words=3):
    # Use the first answer in case there are multiple
    answer = example["answers"]["text"][0]
    return len(answer.strip().split()) <= max_words

def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

def trainModel(model, tokenizer, dataset):
    print("training")
    # Optimizer
    optimizer = AdamW(model.parameters(), lr=5e-5)

    # Training
    model.train()
    batch_num = 0
    totalLoss = 0;
    LossVector = []
    for epoch in range(3):
        small_train = dataset

    # Tokenize the reduced dataset
        tokenized_squad = small_train.map(
            prepare_train_features,
            batched=True,
            remove_columns=small_train.column_names
        )


    # DataLoader
        train_loader = DataLoader(tokenized_squad, batch_size=16, shuffle=True, collate_fn=default_data_collator)
        progress = tqdm(train_loader)
        for batch in progress:
            batch_num += 1

            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            start_positions = batch["start_positions"]
            end_positions = batch["end_positions"]

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                start_positions=start_positions,
                end_positions=end_positions,
            )

            loss = outputs.loss
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            totalLoss += loss.item()
            LossVector.append(loss.item())
            avg = totalLoss / batch_num

            progress.set_description(f"Loss: {avg:.4f}")
    save_path = f"/content/drive/My Drive/Colab Notebooks/Distil-BERT/QA/Model6"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"Model saved at {save_path}")
    print(f"Model 6 Loss Vector: {LossVector}")

In [5]:
!pip install -U -q sentence-transformers   # upgrades to 3.x as of July 2025
import sentence_transformers, os, torch
print("sentence‑transformers version:", sentence_transformers.__version__)

sentence‑transformers version: 5.0.0


In [7]:
!ls -lh "/content/drive/MyDrive/Distil-BERT/1/QA/SentenceTransformer"


total 88M
-rw------- 1 root root  617 Dec 31  1979 config.json
-rw------- 1 root root  205 Dec 31  1979 config_sentence_transformers.json
-rw------- 1 root root  87M Dec 31  1979 model.safetensors
-rw------- 1 root root  349 Dec 31  1979 modules.json
-rw------- 1 root root 9.5K Dec 31  1979 README.md
-rw------- 1 root root   53 Dec 31  1979 sentence_bert_config.json
-rw------- 1 root root  695 Dec 31  1979 special_tokens_map.json
-rw------- 1 root root 1.5K Dec 31  1979 tokenizer_config.json
-rw------- 1 root root 695K Dec 31  1979 tokenizer.json
-rw------- 1 root root 227K Dec 31  1979 vocab.txt


In [8]:
!pip install -U sentence-transformers huggingface-hub


In [ ]:
from sentence_transformers import models, SentenceTransformer
from keybert                import KeyBERT
from transformers           import DistilBertTokenizerFast

ROOT = "/content/drive/MyDrive/Distil-BERT/1/QA"   # ← no space in “MyDrive”

# ── 1. Create an ST model by hand ───────────────────────────────────────
#     • word-embedding module  (your fine-tuned DistilBERT)
#     • pooling module         (add mean-pooling, dimension auto-detected)

bert   = models.Transformer(f"{ROOT}/SentenceTransformer", max_seq_length=512)
pool   = models.Pooling(
           word_embedding_dimension = bert.get_word_embedding_dimension(),  # 768 for DistilBERT
           pooling_mode_mean_tokens = True,
           pooling_mode_cls_token  = False,
           pooling_mode_max_tokens = False
         )
st_embedder = SentenceTransformer(modules=[bert, pool])

# ── 2. Feed that into KeyBERT ───────────────────────────────────────────
kw_model = KeyBERT(model=st_embedder)

# ── 3. Everything else stays the same ───────────────────────────────────
embedder  = SentenceTransformer('all-MiniLM-L6-v2')        # keep if you still need it
path      = f"{ROOT}/Model5"
tokenizer = DistilBertTokenizerFast.from_pretrained(
              f"{ROOT}/Model1",
              local_files_only=True
           )


while True:
    qa_pipeline = pipeline("question-answering", model=path, tokenizer=path)
    question = input("Enter your question: ")
    if question.lower() == 'break':
        break
    else:
        if question.lower() == 'wrong answer':
            dataset = []
            title = ''
            if input("Was the context incorrect?").lower()[0] == 'y':
                if input("Provide correct context?").lower()[0] == 'y':
                    if input("Attach textfile with correct context?").lower()[0] == 'y':
                        uploaded = files.upload()
                        for fn in uploaded.keys():
                            print('User uploaded file "{name}" with length {length} bytes'.format(
                                name=fn, length=len(uploaded[fn])))
                            with open(fn, 'r') as file:
                                dataset += file.readlines()
                    else:
                        dataset += input("CONTEXT: ")
                    for line in dataset:
                        article_text += line
            else:
                dataset.append(article_text)

            if input("Was the title incorrect?").lower()[0] == 'y':
                if input("Provide correct title?").lower()[0] == 'y':
                    title = input("TITLE: ")
            else:
                title = best_title
            answer = input("ANSWER: ")
            if not (title == '' or dataset == []):
                new_example = {
                    "id": "user_example_001",
                    "title": title,
                    "context": article_text,
                    "question": question,
                    "answers": {
                        "text": [answer],
                        "answer_start": [article_text.find(answer)]
                    }
                }
                dataset = Dataset.from_dict({key: [value] for key, value in new_example.items()})

                train_examples = [
                    InputExample(texts=[question, title])
                ]

                # DataLoader
                train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

                # Define loss
                train_loss = losses.MultipleNegativesRankingLoss(kw_model.model.embedding_model)

                # Train the model
                import wandb
                from unittest.mock import patch
                with patch("builtins.input", return_value="1"):
                    wandb.login(anonymous="allow")

                kw_model.model.embedding_model.fit(
                    train_objectives=[(train_dataloader, train_loss)],
                    epochs=5,
                    warmup_steps=100,
                    output_path=f"/content/drive/My Drive/Colab Notebooks/Distil-BERT/QA/SentenceTransformer",

                )

                # Save
                kw_model.model.embedding_model.save(f"/content/drive/My Drive/Colab Notebooks/Distil-BERT/QA/SentenceTransformer")
                trainModel(qa_pipeline.model, tokenizer, dataset)
    print("Q:", question)
    article_text, best_title = get_best_wikipedia_article(extract_keyword_spacy_keybert(question))
    print("A:", answer_question(question, article_text))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Enter your question: How many continents are there?
Q: How many continents are there?
🔎 Candidates: ['Continent', 'Americas', 'Europe', 'Asia', 'Chronology of continents']
✅ Best match: Chronology of continents
A: seven


Device set to use cpu
